Importar librerias libreriales

In [1]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

print('Keras version: ',keras.__version__)

Keras version:  2.5.0


Importar dataset de Bruno

In [3]:
from google.colab import files
uploaded = files.upload()

Saving ybsg5x7-cne.csv to ybsg5x7-cne.csv
Saving xbsg5x7-cne.csv to xbsg5x7-cne.csv


In [4]:
df_X = pd.read_csv("xbsg5x7-cne.csv", header=None)
df_y = pd.read_csv("ybsg5x7-cne.csv", header=None)

y_origen = df_y[0]
y_destino = df_y[1]

X_train, X_test, y_train, y_test = train_test_split(df_X, y_origen, test_size=0.38, random_state=69)

X_test = X_test.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_train_2 = np.delete(X_train,range(35,50),1)
X_train_reshaped = np.expand_dims(X_train_2, axis=2)
print(X_train.shape,X_train_reshaped.shape, y_train.shape)

(5161, 50) (5161, 35, 1) (5161,)


In [6]:
X_train_reshaped.shape = (5161, 5, 7)
X_train_reshaped[0]

array([[0.17894737, 0.03157895, 0.09473684, 0.01052632, 0.02105263,
        0.02105263, 0.11578947],
       [0.24210526, 0.24210526, 0.24210526, 0.24210526, 0.24210526,
        0.10526316, 0.08421053],
       [0.24210526, 0.24210526, 0.24210526, 0.24210526, 0.24210526,
        0.09473684, 0.18947368],
       [0.22105263, 0.21052632, 0.16842105, 0.15789474, 0.06315789,
        0.05263158, 0.04210526],
       [0.13684211, 0.2       , 0.14736842, 0.24210526, 0.23157895,
        0.12631579, 0.07368421]])

In [7]:
#Aquí se obtinenen los atributos "no vectoriales"
XX_train =X_train[0:,36:40]
XX_train.shape

(5161, 4)

In [8]:
y_train[:10]

array([4, 2, 0, 3, 3, 2, 2, 0, 1, 0])

In [9]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
encoder = OneHotEncoder()
y_train_coded = encoder.fit_transform(y_train.reshape(-1,1)).toarray() 

In [12]:
y_train_coded[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

Modelado de la red

In [39]:
convmodel = Sequential()
convmodel.add(Conv1D(filters=50, kernel_size=2, activation='relu', input_shape=(7,1)))
convmodel.add(Conv1D(filters=50, kernel_size=1, activation='relu'))
convmodel.add(Dropout(0.1))
convmodel.add(MaxPooling1D(pool_size=2))

In [40]:
#define two different inputs
input1 = Input((7, 1))
input2 = Input((7, 1))
input3 = Input((7, 1))
input4 = Input((7, 1))
input5 = Input((7, 1))
## falta input 4,5 y el 4 -> 6
input6 = Input((4, ))
   
#use the model to get three different outputs (the same model)
out1 = convmodel(input1)
out2 = convmodel(input2)
out3 = convmodel(input3)
out4 = convmodel(input4)
out5 = convmodel(input5)

#concatenate the outputs and add the final part of your model: 
out = Concatenate()([out1,out2,out3,out4,out5])
out = Flatten()(out)
out = Concatenate()([out,input6])
out = Dense(5000, activation='relu')(out)
convmodel.add(Dropout(0.4))
out = Dense(1000, activation='relu')(out)
convmodel.add(Dropout(0.1))
out = Dense(500, activation='relu')(out)
out = Dense(5, activation='softmax')(out) 

#create the model taking 2 inputs with one output
model = Model([input1,input2,input3,input4,input5,input6],out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.save_weights('model.h5')
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           [(None, 7, 1)]       0                                            
____________________________________________________________________________________________

In [41]:
X_train1 = X_train_reshaped[0:,0]
X_train2 = X_train_reshaped[0:,1]
X_train3 = X_train_reshaped[0:,2]
X_train4 = X_train_reshaped[0:,3]
X_train5 = X_train_reshaped[0:,4]

X_train1.shape=(5161, 7, 1)
X_train2.shape=(5161, 7, 1)
X_train3.shape=(5161, 7, 1)
X_train4.shape=(5161, 7, 1)
X_train5.shape=(5161, 7, 1)

model.load_weights('model.h5')
model.fit([X_train1,X_train2,X_train3,X_train4,X_train5,XX_train], y_train_coded, epochs=100, batch_size=64, verbose=True)

Epoch 1/100
81/81 [==============================] - 13s 116ms/step - loss: 1.5152 - accuracy: 0.2935
Epoch 2/100
81/81 [==============================] - 9s 115ms/step - loss: 1.3981 - accuracy: 0.3409
Epoch 3/100
81/81 [==============================] - 9s 116ms/step - loss: 1.3455 - accuracy: 0.4026
Epoch 4/100
81/81 [==============================] - 9s 116ms/step - loss: 1.2727 - accuracy: 0.4436
Epoch 5/100
81/81 [==============================] - 9s 115ms/step - loss: 1.2146 - accuracy: 0.4766
Epoch 6/100
81/81 [==============================] - 9s 115ms/step - loss: 1.1573 - accuracy: 0.5023
Epoch 7/100
81/81 [==============================] - 9s 115ms/step - loss: 1.0958 - accuracy: 0.5291
Epoch 8/100
81/81 [==============================] - 9s 116ms/step - loss: 1.0314 - accuracy: 0.5585
Epoch 9/100
81/81 [==============================] - 9s 116ms/step - loss: 1.0306 - accuracy: 0.5582
Epoch 10/100
81/81 [==============================] - 9s 114ms/step - loss: 0.9635 - accur

In [42]:
#X_test = X_test.to_numpy()
X_test_2 = np.delete(X_test,range(35,50),1)
X_test_reshaped = np.expand_dims(X_test_2, axis=2)
print(X_test_reshaped.shape)
X_test_reshaped.shape = (3164, 5, 7)
XX_test = X_test[0:,36:40]

X_test1 = X_test_reshaped[0:,0]
X_test2 = X_test_reshaped[0:,1]
X_test3 = X_test_reshaped[0:,2]
X_test4 = X_test_reshaped[0:,3]
X_test5 = X_test_reshaped[0:,4]

(3164, 35, 1)


In [43]:
m=model.predict([X_test1, X_test2, X_test3, X_test4, X_test5, XX_test])


In [44]:
import sklearn.metrics as skm
#print(m)
m_final = np.argmax(m, axis=1)
print(m_final)
print(skm.accuracy_score(y_test, m_final))
print(skm.f1_score(y_test, m_final, average='macro'))
print(skm.mean_absolute_error(y_test, m_final))

[4 3 2 ... 4 1 0]
0.5508849557522124
0.5493838033052387
0.9190897597977244
